In [1]:
import numpy as np
import cv2 as cv

In [2]:
# Function to read the secret image
def read_image(image_name):
    image = cv.imread(image_name)
    print("The shape of the secret image is = ", image.shape)
    resized_image = cv.resize(image, (500,500))
    cv.imshow("Secret Image",resized_image)
    cv.waitKey(0)
    return image

In [3]:
# Function to generate a random key
def generateRandomKey(key_shape):
    dealer_key = np.zeros(key_shape).astype(int)
    for i in range(0, len(dealer_key)):
        for j in range(0, len(dealer_key[i])):
            for k in range(0, 3):
                dealer_key[i][j][k] = (np.random.randint(0,255))
    return dealer_key


In [4]:
# Function to convert the secret image into encrypted image
def convertToEncryptedImage(image, random_key):
    print("Converting to Encrypted Image...")
    encrypted_image = np.zeros(image.shape).astype(int)
    
    for i in range(0, len(random_key)):
        for j in range(0, len(random_key[i])):
            for k in range(0, 3):
                encrypted_image[i][j][k] = image[i][j][k]^random_key[i][j][k]
    
    return encrypted_image

In [5]:
# Function to generate unique id's for each participant
def getUniqueIds(n):
    temp_arr = np.zeros(n).astype(int)
    for i in range(n):
        temp_arr[i] = np.random.randint(0,255)
    return temp_arr

In [6]:
#Function to get encrypted id's for each participant
def getEncryptedIds(unique_ids, n):
    temp_arr = []
    for i in range(n):
        x = np.zeros(8).astype(int)
        s = '{0:08b}'.format(unique_ids[i])
        msb = s[0:4]
        lsb = s[4:8]
        for i in range(4):
            x[i] = int(msb[i])
        for i in range(4):
            x[i+4] = int(msb[i])^int(lsb[i])
        res = int("".join(str(l) for l in x), 2)
        temp_arr.append(res)
        
    return temp_arr

In [7]:
# Left circular rotation of an array str -> x times
def leftRotate(bit_8_s, x):
    temp_str = []
    for i in range(x):
        temp_str.append(bit_8_s[i])
    check_point = 0
    for i in range(0, len(bit_8_s)-x):
        bit_8_s[i] = bit_8_s[i+x]
        check_point = i
    for i in range(0,x):
        bit_8_s[check_point+1+i] = temp_str[i]
    return bit_8_s

In [8]:
#Function to perform left circular shift
def leftCircularShift(R1, x):

    for i in range(0, len(R1)):
        for j in range(0, len(R1[i])):
            for k in range(0, 3):
                s = '{0:08b}'.format(R1[i][j][k])
                temp_s = np.zeros(8).astype(int)
                for l in range(8):
                    temp_s[l] = int(s[l])
                x = x%8
                temp_s = leftRotate(temp_s, x)
                res = int("".join(str(l) for l in temp_s), 2)
                R1[i][j][k] = res
    
    return R1

In [9]:
# Right circular rotation of an array str -> x times
def rightRotate(bit_8_s, x):
    temp_str = []
    for i in range(len(bit_8_s)-x,len(bit_8_s)):
        temp_str.append(bit_8_s[i])
    for i in range(len(bit_8_s)-1, x-1, -1):
        bit_8_s[i] = bit_8_s[i-x]
    for i in range(0,x):
        bit_8_s[i] = temp_str[i]
    return bit_8_s

In [10]:
#Function to perform right circular shift
def rightCircularShift(R1, x):

    for i in range(0, len(R1)):
        for j in range(0, len(R1[i])):
            for k in range(0, 3):
                s = '{0:08b}'.format(R1[i][j][k])
                temp_s = np.zeros(8).astype(int)
                for l in range(8):
                    temp_s[l] = int(s[l])
                x = x%8
                temp_s = rightRotate(temp_s, x)
                res = int("".join(str(l) for l in temp_s), 2)
                R1[i][j][k] = res
    
    return R1

In [11]:
N = input('Enter the total number of shares (N) : ')
N = int(N)
print("The total number of shares (N) = ", N)
K = input("Enter the number of shares required to retreive the secret image (K) :")
K = int(K)
print("The number of shares required to retreive the secret image (K) = ", K)
image_name = input("Enter the name of the image (with extension)")
image = read_image(image_name)
random_key = generateRandomKey(image.shape)
print("The shape of random key = ", random_key.shape)
encrypted_image = convertToEncryptedImage(image, random_key)
print("The shape of encrypted image = ", encrypted_image.shape)
unique_ids = getUniqueIds(N)
print("The unique id's of all the N participants are :")
print(unique_ids)

The total number of shares (N) =  10
The number of shares required to retreive the secret image (K) =  5
The shape of the secret image is =  (1225, 840, 3)
The shape of random key =  (1225, 840, 3)
Converting to Encrypted Image...
The shape of encrypted image =  (1225, 840, 3)
The unique id's of all the N participants are :
[229 254 164 158 188 163 177 129  69 201]


In [12]:
# Function to get authenticated image for each participant (R1)
def getR1(encrypted_image, shares):
    R1 = np.zeros(encrypted_image.shape).astype(int)
    R1_remainder = np.zeros(encrypted_image.shape).astype(int)
    
    for i in range(0, len(R1)):
        for j in range(0, len(R1[i])):
            for k in range(0, 3):
                R1[i][j][k] = int(encrypted_image[i][j][k]//int(shares))
                R1_remainder[i][j][k] = encrypted_image[i][j][k]%int(shares)
    
    return [R1, R1_remainder]

authenticated_image = getR1(encrypted_image, K) # array of R1 and R1_remainder 
print(authenticated_image[0].shape, authenticated_image[1].shape)

(1225, 840, 3) (1225, 840, 3)


In [13]:
encrypted_ids = getEncryptedIds(unique_ids, N)
print("The encrypted ids are : ")
print(encrypted_ids)

The encrypted ids are : 
[235, 241, 174, 151, 183, 169, 186, 137, 65, 197]


In [14]:
#generating n shares
def generate_N_shares(R1, n, encrypted_ids):
    S = []
    for i in range(0,n):
        temp = R1
        temp_S = leftCircularShift(R1, encrypted_ids[i])
        S.append(temp_S)
    return S


In [15]:

shares = generate_N_shares(authenticated_image[0], N, encrypted_ids)


In [16]:
# saving first k shares
def saveKshares(S, k):
    for i in range(0, k):
        filename = "share_"+str(i+1)+".png"
        cv.imwrite(filename, S[i])
saveKshares(shares, K)

10 <class 'str'>
